In [1]:
import boto3

In [2]:
client = boto3.client("sagemaker")

In [8]:
training_job = client.describe_training_job(TrainingJobName="sagemaker-pytorch-2021-01-16-17-49-15-976")

In [11]:
training_job

{'TrainingJobName': 'sagemaker-pytorch-2021-01-16-17-49-15-976',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:977053370764:training-job/sagemaker-pytorch-2021-01-16-17-49-15-976',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-17-49-15-976/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'sagemaker_container_log_level': '20',
  'sagemaker_job_name': '"sagemaker-pytorch-2021-01-16-17-49-15-976"',
  'sagemaker_program': '"train.py"',
  'sagemaker_region': '"us-east-1"',
  'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-17-49-15-976/source/sourcedir.tar.gz"'},
 'AlgorithmSpecification': {'TrainingImage': '520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-pytorch:0.4.0-gpu-py3',
  'TrainingInputMode': 'File',
  'EnableSageMakerMetricsTimeSeries': False},
 'RoleArn': 'arn:aws:iam::977053370764:role/service-role

In [9]:
model_details = client.describe_model(ModelName="sagemaker-pytorch-2021-01-16-18-02-29-843")

In [10]:
model_details["PrimaryContainer"]

{'Image': '520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-pytorch:0.4.0-cpu-py3',
 'Mode': 'SingleModel',
 'ModelDataUrl': 's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-17-49-15-976/output/model.tar.gz',
 'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
  'SAGEMAKER_PROGRAM': 'train.py',
  'SAGEMAKER_REGION': 'us-east-1',
  'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-17-49-15-976/source/sourcedir.tar.gz'}}

In [15]:
training_job["ModelArtifacts"]["S3ModelArtifacts"]

's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-17-49-15-976/output/model.tar.gz'

In [28]:
prefx_name = "udacity1"
model_name = f"{prefx_name}-model"
confg_name = f"{prefx_name}-config"
endpt_name = f"{prefx_name}-endpoint"

In [29]:
response = client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'ContainerHostname': prefx_name,
        'Image': training_job["AlgorithmSpecification"]["TrainingImage"],
        'ImageConfig': {
            'RepositoryAccessMode': 'Platform'
        },
        'Mode': 'SingleModel',
        'ModelDataUrl': training_job["ModelArtifacts"]["S3ModelArtifacts"]
    },
    ExecutionRoleArn='arn:aws:iam::977053370764:role/service-role/AmazonSageMaker-ExecutionRole-20201202T141643')

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:us-east-1:977053370764:model/udacity1-model".

In [30]:
response = client.create_endpoint_config(
    EndpointConfigName=confg_name,
    ProductionVariants=[
        {
            'VariantName': 'string',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium'
        },
    ]
)

In [31]:
response = client.create_endpoint(
    EndpointName=endpt_name,
    EndpointConfigName=confg_name
)

In [45]:
import os
import io
import pandas as pd

In [36]:
ibovespa = pd.read_csv(os.path.join("data", "data.csv"))

In [39]:
with open(os.path.join("data", "data.csv")) as f:
    dados = f.read().encode("utf-8")

In [48]:
df = pd.read_csv(  io.StringIO(dados.decode('utf-8'))  , sep=",")

In [49]:
df

,symbol,date,open,close,high,low,volume,adjclose
0,^BVSP,2019-07-26,102654.0,102819.0,103209.0,102196.0,5141400.0,102819.0
1,^BVSP,2019-07-29,102817.0,103483.0,103483.0,102461.0,3559900.0,103483.0
2,^BVSP,2019-07-30,103483.0,102933.0,103555.0,102596.0,4060500.0,102933.0
3,^BVSP,2019-07-31,102946.0,101812.0,103129.0,100950.0,5072900.0,101812.0
4,^BVSP,2019-08-01,101819.0,102126.0,104056.0,101819.0,7129100.0,102126.0
...,...,...,...,...,...,...,...,...
360,^BVSP,2021-01-11,125075.0,122807.0,125075.0,122506.0,9537600.0,122807.0
361,^BVSP,2021-01-12,123255.0,123998.0,124584.0,123227.0,8949000.0,123998.0
362,^BVSP,2021-01-13,123996.0,122040.0,124032.0,121016.0,10291500.0,122040.0
363,^BVSP,2021-01-14,121947.0,123481.0,123896.0,121947.0,8974400.0,123481.0
